In [1]:
from algorithm.utils.data.tf.dataimage import DataImage
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import keras
%load_ext tensorboard

In [2]:
dataset = DataImage(data_path="Input/dataset", split=0.2, resize=True, height='auto', width='auto', normalize=True, mean='auto', name="Electronic components dataset", batch_size=16)

In [3]:
dataset

Electronic components dataset
	Number of points: 2884
	Root location: Input/dataset
	Split: 0.2
	Transform used:	Resize(size=(128, 128)
	Normalize(mean=[0.00392156862745098, 0.00392156862745098, 0.00392156862745098], std=None)


In [4]:
trainset = dataset.get_set(split="train")
validset = dataset.get_set(split="validation")

In [5]:
# get some random training images
dataiter = iter(trainset)
images, labels = next(dataiter)

In [6]:
images.shape, len(labels)

(TensorShape([16, 128, 128, 3]), 16)

In [7]:
images

<tf.Tensor: shape=(16, 128, 128, 3), dtype=float32, numpy=
array([[[[0.06666667, 0.16078432, 0.10588236],
         [0.06666667, 0.16078432, 0.10588236],
         [0.06666667, 0.16078432, 0.10588236],
         ...,
         [0.07450981, 0.16862746, 0.1137255 ],
         [0.07450981, 0.16862746, 0.1137255 ],
         [0.07450981, 0.16862746, 0.1137255 ]],

        [[0.06666667, 0.16078432, 0.10588236],
         [0.06666667, 0.16078432, 0.10588236],
         [0.06666667, 0.16078432, 0.10588236],
         ...,
         [0.07450981, 0.16862746, 0.1137255 ],
         [0.07450981, 0.16862746, 0.1137255 ],
         [0.07450981, 0.16862746, 0.1137255 ]],

        [[0.06666667, 0.16078432, 0.10588236],
         [0.06666667, 0.16078432, 0.10588236],
         [0.06666667, 0.16078432, 0.10588236],
         ...,
         [0.07450981, 0.16862746, 0.1137255 ],
         [0.07450981, 0.16862746, 0.1137255 ],
         [0.07450981, 0.16862746, 0.1137255 ]],

        ...,

        [[0.06666667, 0.16078432,

In [8]:
labels

<tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'Rectangular pin', b'Ceramic capacitor', b'Rectangular pin',
       b'Resistor', b'Ceramic capacitor', b'Ceramic capacitor',
       b'Not mounted', b'SOT', b'Rectangular pin', b'Ceramic capacitor',
       b'Rectangular pin', b'Not mounted', b'Rectangular pin',
       b'Ceramic capacitor', b'Jumper', b'TO252'], dtype=object)>

In [9]:
class_names = dataset.labels

In [10]:
class_names

['Pushbutton',
 'Not mounted',
 'Connector',
 'Oscillator',
 'Diode',
 'Jumper cut',
 'Resistor Network',
 'Optoisolator',
 'Rectangular pin',
 'IC',
 'Tantalum capacitor',
 'Ceramic capacitor',
 'Dip Switch',
 'TO252',
 'Fuse',
 'Resistor',
 'Relais',
 'LED',
 'Inductor',
 'SOT',
 'Jumper',
 'Minimelf']

In [ ]:
import matplotlib.pyplot as plt
image_batch, label_batch = next(iter(trainset))

plt.figure(figsize=(60, 60))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8")) # to be implemented: normalization and resize
    label = label_batch[i].numpy().decode("utf-8")
    plt.title(label, fontsize=60)
    plt.axis("off")

In [12]:
from algorithm.deep.tf.neural_networks import ResNet2__0

In [13]:
trainset = dataset.apply_one_hot_encoding(trainset)
validset = dataset.apply_one_hot_encoding(validset)

In [14]:
out_dir = "out/models/ResNet2__0_2"

In [15]:
model = ResNet2__0(len(class_names),input_shape=(128, 128, 3))
tb_callback = keras.callbacks.TensorBoard(log_dir=f'{out_dir}/tensorboard_log', histogram_freq=1)

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
    run_eagerly=True,
)

In [17]:
tf.get_logger().setLevel('ERROR')

model.fit(trainset, validation_data=validset, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
145/145 [==============================] - 138s 728ms/step - loss: 3.8069 - accuracy: 0.6937 - val_loss: 79.8402 - val_accuracy: 0.1597
Epoch 2/1000
145/145 [==============================] - 95s 654ms/step - loss: 0.4235 - accuracy: 0.8934 - val_loss: 105.3690 - val_accuracy: 0.1597
Epoch 3/1000
145/145 [==============================] - 92s 633ms/step - loss: 0.3308 - accuracy: 0.9207 - val_loss: 9.2799 - val_accuracy: 0.0226
Epoch 4/1000
145/145 [==============================] - 92s 632ms/step - loss: 0.2279 - accuracy: 0.9536 - val_loss: 4.8547 - val_accuracy: 0.0122
Epoch 5/1000
145/145 [==============================] - 92s 630ms/step - loss: 0.2166 - accuracy: 0.9519 - val_loss: 3.6459 - val_accuracy: 0.2326
Epoch 6/1000
145/145 [==============================] - 92s 635ms/step - loss: 0.2153 - accuracy: 0.9601 - val_loss: 2.0395 - val_accuracy: 0.4896
Epoch 7/1000
145/145 [==============================] - 91s 629ms/step - loss: 0.3869 - accuracy: 0.9328 - val_los

In [18]:
model.summary()

Model: "res_net2__0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 4, 4, 2048)        58370944  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 1000)              32769000  
                                                                 
 dense_1 (Dense)             (None, 22)                22022     
                                                                 
 model (Functional)          (None, 22)                91161966  
                                                                 
Total params: 91161966 (347.76 MB)
Trainable params: 91010542 (347.18 MB)
Non-trainable params: 151424 (591.50 KB)
_________________________________________________________________


In [19]:
model.save(out_dir)

In [ ]:
keras.utils.plot_model(model, f'{out_dir}/ResNet2__0_2.svg', show_shapes=True, show_layer_activations=True, show_layer_names=True)

In [21]:
# Evaluation on unseen data.


test_loss, test_acc = model.evaluate(validset, verbose=2)
print('\nTest accuracy:', test_acc)

36/36 - 5s - loss: 1.3107 - accuracy: 0.9809 - 5s/epoch - 142ms/step

Test accuracy: 0.9809027910232544
